# Setup Variables and home directory

MNIST, FashionMNIST, GTSRB, Cifar10

In [1]:
DATASET = 'GTSRB'
POSTFIX = 'AdamW-32-30'
FILENAME_POSTFIX = f'{DATASET}_{POSTFIX}'
SEED = 42
CUDA = 0
GPU_NAME = f'cuda:{CUDA}'

In [2]:
# set homw directory
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

base

PosixPath('/home/ah19/runtime-monitoring')

In [3]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Libraries

In [4]:
import pandas as pd
import numpy as np

from fastprogress import progress_bar

from utilities.utils import load_json, load_pickle
from utilities.pathManager import fetchPaths
from utilities.MonitorUnifiedBDD import MonitorBDD

# Paths

In [5]:
paths = fetchPaths(base, DATASET, POSTFIX, False)

path_data = paths['data']
path_lhl = paths['lhl']
path_lhl_raw = paths['lhl_raw']
path_lhl_pca = paths['lhl_pca']

# configs = load_json(paths['configuration'])
# config = configs['configuration']
# model_setup = configs['model_setup']
# model_config = configs['model_config']
# optim_name = list(config['optimizer'].keys())[0]
# optim_args = config['optimizer'][optim_name]
# scheduler_name = list(config['scheduler'].keys())[0]
# scheduler_args = config['scheduler'][scheduler_name]

# Seed

In [6]:
np.random.seed(SEED)

# Import Data

In [7]:
FLAVOR = 'raw'

if FLAVOR == 'raw':
    path_lhl = path_lhl_raw
else: path_lhl = path_lhl_pca

# import Data
df_train = pd.read_csv(path_lhl / f"{FILENAME_POSTFIX}_{FLAVOR}_train.csv")

# select only true classified
df_true = df_train[df_train["true"] == True].copy()
df_true = df_true.drop("true", axis=1).reset_index(drop=True)

df_test = pd.read_csv(path_lhl / f"{FILENAME_POSTFIX}_{FLAVOR}_test.csv")

# Shuffel and Split indecies

In [8]:
periods = 10

# shuffle data
schuffled_index = df_true.index.to_numpy().copy()
np.random.shuffle(schuffled_index)

# split data into batches
fragments_index = np.array_split(schuffled_index, periods)

# print shape
schuffled_index.shape, np.concatenate(fragments_index).shape

((38888,), (38888,))

# Build BDD

In [9]:
# each set of data has its own threshold
thld_p = 0.9
eta = 0
neurons = []


# save scores and stats
stats = pd.DataFrame()
train_scores = pd.DataFrame()
test_scores = pd.DataFrame()

# progress bar
pb = progress_bar(range(len(fragments_index)))

# buidl for each fragment
for fragment in pb:
    # indecies
    ix = np.concatenate(fragments_index[0:fragment+1])

    # calculate threshold
    thld = np.quantile( df_true.loc[ix].drop('y', axis=1), thld_p)
    
    # BDD
    patterns = MonitorBDD( df_true.loc[ix].shape[1]-1, thld, neurons=neurons)
    df_train_copy, df_test_copy = patterns.add_dataframe( df_true.loc[ix], eta, eval_dfs=[df_train.loc[ix].copy(), df_test.copy()] )

    
    # score
    df_train_scores = patterns.score_dataframe_multi_eta(df_train_copy, eta)
    df_test_scores = patterns.score_dataframe_multi_eta(df_test_copy, eta)

    
    # add metadata
    patterns.stats['thld'] = f'qth_{thld_p}'
    patterns.stats['period'] = fragment
    patterns.stats['num_observations'] = ix.shape[0]
    df_train_scores['period'] = fragment
    df_train_scores['num_observations'] = ix.shape[0]
    df_test_scores['period'] = fragment
    df_test_scores['num_observations'] = ix.shape[0]
    
    
    # collect scores
    stats = pd.concat([stats, patterns.stats])
    train_scores = pd.concat([train_scores, df_train_scores])
    test_scores = pd.concat([test_scores, df_test_scores])

In [10]:
stats

,thld,eta,build_time,size_mb,reorder_time,num_patterns,num_unique_patterns,num_reorder,num_neurons,start_time,end_time,period,num_observations
1,qth_0.9,0.0,0.001,4304.379,0.0,3889.0,1328.0,0.0,30.0,2023-07-15 11:13:25,2023-07-15 11:13:25,0,3889
1,qth_0.9,0.0,0.002,4304.479,0.0,7778.0,1926.0,0.0,30.0,2023-07-15 11:13:28,2023-07-15 11:13:28,1,7778
1,qth_0.9,0.0,0.002,4304.604,0.0,11667.0,2384.0,0.0,30.0,2023-07-15 11:13:31,2023-07-15 11:13:31,2,11667
1,qth_0.9,0.0,0.003,4304.573,0.0,15556.0,2765.0,0.0,30.0,2023-07-15 11:13:33,2023-07-15 11:13:34,3,15556
1,qth_0.9,0.0,0.004,4304.584,0.0,19445.0,3040.0,0.0,30.0,2023-07-15 11:13:37,2023-07-15 11:13:37,4,19445
1,qth_0.9,0.0,0.004,4304.598,0.0,23334.0,3337.0,0.0,30.0,2023-07-15 11:13:40,2023-07-15 11:13:40,5,23334
1,qth_0.9,0.0,0.005,4304.578,0.0,27223.0,3587.0,0.0,30.0,2023-07-15 11:13:43,2023-07-15 11:13:43,6,27223
1,qth_0.9,0.0,0.005,4304.745,0.0,31112.0,3797.0,0.0,30.0,2023-07-15 11:13:47,2023-07-15 11:13:47,7,31112
1,qth_0.9,0.0,0.006,4305.081,0.0,35000.0,3980.0,0.0,30.0,2023-07-15 11:13:50,2023-07-15 11:13:51,8,35000
1,qth_0.9,0.0,0.006,4304.747,0.0,38888.0,4178.0,0.0,30.0,2023-07-15 11:13:54,2023-07-15 11:13:55,9,38888


In [11]:
train_scores.loc[ train_scores['y'] == 'all' ]

,y,count,false,false_misclassified,false_classified,outOfPattern,outOfPatternMisclassified,outOfPatternClassified,eta,period,num_observations
43,all,3889.0,714.0,18.0,696.0,0.183595,0.025210,0.974790,0,0,3889
43,all,7778.0,875.0,25.0,850.0,0.112497,0.028571,0.971429,0,1,7778
43,all,11667.0,873.0,32.0,841.0,0.074826,0.036655,0.963345,0,2,11667
43,all,15556.0,795.0,42.0,753.0,0.051106,0.052830,0.947170,0,3,15556
43,all,19445.0,722.0,47.0,675.0,0.037130,0.065097,0.934903,0,4,19445
43,all,23334.0,641.0,52.0,589.0,0.027471,0.081123,0.918877,0,5,23334
43,all,27223.0,533.0,53.0,480.0,0.019579,0.099437,0.900563,0,6,27223
43,all,31112.0,361.0,55.0,306.0,0.011603,0.152355,0.847645,0,7,31112
43,all,35000.0,226.0,65.0,161.0,0.006457,0.287611,0.712389,0,8,35000
43,all,38888.0,74.0,74.0,0.0,0.001903,1.000000,0.000000,0,9,38888


In [12]:
test_scores.loc[ test_scores['y'] == 'all' ]

,y,count,false,false_misclassified,false_classified,outOfPattern,outOfPatternMisclassified,outOfPatternClassified,eta,period,num_observations
43,all,4192.0,1232.0,38.0,1194.0,0.293893,0.030844,0.969156,0,0,3889
43,all,4192.0,949.0,24.0,925.0,0.226384,0.025290,0.974710,0,1,7778
43,all,4192.0,794.0,23.0,771.0,0.189408,0.028967,0.971033,0,2,11667
43,all,4192.0,686.0,21.0,665.0,0.163645,0.030612,0.969388,0,3,15556
43,all,4192.0,644.0,18.0,626.0,0.153626,0.027950,0.972050,0,4,19445
43,all,4192.0,590.0,18.0,572.0,0.140744,0.030508,0.969492,0,5,23334
43,all,4192.0,551.0,18.0,533.0,0.131441,0.032668,0.967332,0,6,27223
43,all,4192.0,507.0,15.0,492.0,0.120945,0.029586,0.970414,0,7,31112
43,all,4192.0,486.0,15.0,471.0,0.115935,0.030864,0.969136,0,8,35000
43,all,4192.0,464.0,15.0,449.0,0.110687,0.032328,0.967672,0,9,38888
